In [2]:
import pandas as pd

In [46]:
compustat = pd.read_csv(r"C:\Etudes\EPFL\1ère\MA2\ML\PROJECT\data_raw\Predictors\CompFirmCharac.csv\CompFirmCharac.csv")

C:\Users\rvent\AppData\Local\Temp\ipykernel_13920\3602216866.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  compustat = pd.read_csv(r"C:\Etudes\EPFL\1ère\MA2\ML\PROJECT\data_raw\Predictors\CompFirmCharac.csv\CompFirmCharac.csv")


In [45]:
compustat.head()

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,xoptepsqpy,xoptepsy,xoptqpy,xopty,xrdy,xsgay,exchg,cik,costat,fic
0,1000,1966-03-31,1966,1.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
1,1000,1966-06-30,1966,2.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
2,1000,1966-09-30,1966,3.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
3,1000,1966-12-31,1966,4.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
4,1000,1967-03-31,1967,1.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA


In [ ]:
# Possibility to check the 10Q or 10K fillinf date?
#Eg If I have a datadate 1966.03.31, it  means that this financial statement cover the quarter that ended 
# at this date
#So the associated information will be avaialable after a specfic lag. 


(2052014, 256)

In [17]:
import pandas as pd

def identify_string_and_mixed_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return a DataFrame listing each column that contains non-numeric (object) or mixed-type values,
    along with its pandas dtype and a sample of Python types observed.
    """
    result = []
    for col in df.columns:
        if df[col].dtype == 'object':
            # collect up to 5 unique Python types in this column
            sample_types = df[col].dropna().map(type).unique()[:5].tolist()
            result.append({
                'column': col,
                'pandas_dtype': df[col].dtype,
                'sample_types': sample_types
            })
    return pd.DataFrame(result)

# Usage:
mixed_df = identify_string_and_mixed_columns(compustat)
print(mixed_df.to_string(index=False))





  column pandas_dtype    sample_types
  curcdq       object [<class 'str'>]
datacqtr       object [<class 'str'>]
datafqtr       object [<class 'str'>]
     fic       object [<class 'str'>]


In [19]:
from pandas import DataFrame

def value_counts_for_column(df: DataFrame, column: str) -> DataFrame:
    """
    Return a DataFrame listing each unique value in the specified column of df
    along with its occurrence count, sorted by count descending.

    """
    counts = df[column].value_counts(dropna=False)
    return counts.rename_axis('value').reset_index(name='count')


result = value_counts_for_column(compustat, 'fic')

result.head()
# print(result)
#SONGER A HOT ENCODER FIC, INCORPORATION DE BASE DU PAYS PEU ETRE INTERESSANT


,value,count
0,USA,621058
1,CAN,132624
2,CYM,13840
3,ISR,7695
4,GBR,7165


In [21]:


result = value_counts_for_column(compustat, 'curcdq')

result.head()
# print(result)

#CURRENCY DANS LAQUELLE LA FINANCIAL DATA EST DISPONIBLE 

,value,count
0,USD,724391
1,CAD,119439


In [22]:
result = value_counts_for_column(compustat, 'datafqtr')

result.head()

,value,count
0,2024Q2,11963
1,2024Q3,11880
2,2024Q1,11857
3,2023Q4,11735
4,2023Q3,11637


In [23]:
result = value_counts_for_column(compustat, 'datacqtr')

result.head()

,value,count
0,2024Q2,11943
1,2024Q3,11895
2,2024Q1,11841
3,2023Q4,11736
4,2023Q3,11615


In [ ]:
#Preliminary cleaning:

#1) Keep only industrial firm (avoid distortion of classic ratio and consistency with benchmark studies as JKP)
compustat = compustat[compustat['indfmt'] == 'INDL'].drop(columns='indfmt')
#Drop it

#2) Keep only consolidated financial statement to avoid double counting
compustat = compustat[compustat['consol'] == 'C'].drop(columns='consol')
#drop it

#3) Keep only GAAP Standard format: Ensure consistency
compustat = compustat[compustat['datafmt'] == 'STD'].drop(columns='datafmt')
#drop it


#4) Check if there are any duplicate  pair (gvkey, datadate):
#A voir si on drop les duplicate keys -> Ca fou la merde dans le ML sinon on a quand meme 1550 lignes avec la clef dupliquée.

#5) Lag the features A VOIR SI LE DATASET EST DEJA LAGGE EN CE SENS (EN ATTENTE DU GIUS SUR ED) SI PAS LAGGUE DECALLE DE 3 MOIS

# 6) Convert data to datetime:
compustat['datadate'] = pd.to_datetime(compustat['datadate'])

#7) Drop raw identifier columns:
compustat = compustat.drop(columns=['tic','cusip','conm'])

#8) Keep only active firms
compustat = compustat[compustat['costat'] == 'A'].drop(columns='costat')

#9) Drop datacqtr and  datafqtr (we apply 3 month conservative lag)  ON APPLIQUE OU PAS ( GIUS???)
compustat = compustat.drop(columns=['datacqtr','datafqtr'])

In [52]:
#Build a function where we check wheter a collumn is always the same for all the dataset

def print_constant_columns(df):
    """
    Print all columns of the DataFrame that contain a single unique value (including NaN).
    
    For each such column, outputs the column name and the constant value it holds.
    """
    
    const_series = df.loc[:, df.nunique(dropna=False) == 1].iloc[0]
    print(const_series.to_string())

# Usage:
print_constant_columns(compustat)


Series([], )


In [51]:
#WORK ON COLLUMNS

#8) Drop all remaining columns that have exactly one unique value
compustat = compustat.drop(
    [col for col in compustat.columns if compustat[col].nunique(dropna=False) == 1],
    axis=1
)

In [53]:
def count_high_nan_columns(df, threshold):
    """
    Count how many columns in df have more than `threshold` fraction of missing values.
    
    Parameters
    ----------
    df : DataFrame
        The DataFrame to inspect.
    threshold : float, default 0.7
        Fraction of rows that must be NaN to count the column.
    
    Returns
    -------
    int
        Number of columns with > threshold fraction of NaNs.
    """
    return (df.isna().mean() > threshold).sum()

count_high_nan_columns(compustat, 0.7)

np.int64(154)

In [54]:

def count_nan_columns_by_threshold(df: pd.DataFrame, thresholds=None) -> dict:
    """
    Count how many columns in `df` have a proportion of NaNs strictly greater than each threshold.

    """
    if thresholds is None:
        thresholds = [0.95, 0.90, 0.85, 0.80, 0.75, 0.70, 0.60, 0.50, 0.40, 0.30,0.20,0.10]
    na_frac = df.isna().mean()
    return {thr: int((na_frac >= thr).sum()) for thr in thresholds}

# Usage example:
counts = count_nan_columns_by_threshold(compustat)
print(counts)


{0.95: 87, 0.9: 117, 0.85: 127, 0.8: 145, 0.75: 152, 0.7: 154, 0.6: 158, 0.5: 176, 0.4: 189, 0.3: 232, 0.2: 238, 0.1: 238}


In [55]:
#Drop all the collumns with missing value >= 70%
compustat = compustat.loc[:, compustat.isnull().mean(axis=0) <= 0.7]

In [56]:
counts = count_nan_columns_by_threshold(compustat)
print(counts)

{0.95: 0, 0.9: 0, 0.85: 0, 0.8: 0, 0.75: 0, 0.7: 0, 0.6: 4, 0.5: 22, 0.4: 35, 0.3: 78, 0.2: 84, 0.1: 84}


In [57]:
def count_nan_rows_by_threshold(df: pd.DataFrame, thresholds=None) -> dict:
    """
    Count how many rows in `df` have a proportion of NaNs greater than or equal to each threshold.


    """
    if thresholds is None:
        thresholds = [0.95, 0.90, 0.85, 0.80, 0.75, 0.70, 0.60, 0.50, 0.40, 0.30, 0.20, 0.10]
    na_frac_rows = df.isna().mean(axis=1)
    return {thr: int((na_frac_rows >= thr).sum()) for thr in thresholds}

# Usage:
counts_rows = count_nan_rows_by_threshold(compustat)
print(counts_rows)


{0.95: 0, 0.9: 198997, 0.85: 224782, 0.8: 246397, 0.75: 247206, 0.7: 250941, 0.6: 277805, 0.5: 317721, 0.4: 339851, 0.3: 359346, 0.2: 425616, 0.1: 503681}


In [58]:
na_frac_rows = compustat.isna().mean(axis=1)
# Look at the 10 rows with the highest NaN fraction:
worst10 = na_frac_rows.sort_values(ascending=False).head(10).index
print(compustat.loc[worst10, compustat.columns[compustat.iloc[0].isna()]].head(10))

#Still have a lot of rows with empty nans.. lots of collumn may be really industry specific ... problematic

         acchgy  aolochy  apalchy  aqcy  capxy  chechy  cibegniy  cicurry  \
1878709     NaN      NaN      NaN   NaN    NaN     NaN       NaN      NaN   
1878678     NaN      NaN      NaN   NaN    NaN     NaN       NaN      NaN   
1878679     NaN      NaN      NaN   NaN    NaN     NaN       NaN      NaN   
1878680     NaN      NaN      NaN   NaN    NaN     NaN       NaN      NaN   
1878710     NaN      NaN      NaN   NaN    NaN     NaN       NaN      NaN   
1878681     NaN      NaN      NaN   NaN    NaN     NaN       NaN      NaN   
1907456     NaN      NaN      NaN   NaN    NaN     NaN       NaN      NaN   
1907457     NaN      NaN      NaN   NaN    NaN     NaN       NaN      NaN   
1907458     NaN      NaN      NaN   NaN    NaN     NaN       NaN      NaN   
1907459     NaN      NaN      NaN   NaN    NaN     NaN       NaN      NaN   

         cidergly  cimiiy  ...  txachy  txbcofy  txbcoy  txdcy  txpdy  txwy  \
1878709       NaN     NaN  ...     NaN      NaN     NaN    NaN    NaN   N

In [59]:
#Check how many duplicate pairs there are
duplicate_counts = (
    compustat
    .groupby(['gvkey', 'datadate'])
    .size()
    .reset_index(name='count')
)

# Keep only those with count > 1
duplicate_pairs = duplicate_counts[duplicate_counts['count'] > 1]

print(duplicate_pairs)

         gvkey   datadate  count
1192      1076 1995-03-31      2
2293      1117 1990-03-31      2
2661      1161 1987-03-31      2
3708      1224 2013-12-31      2
3709      1224 2014-03-31      2
...        ...        ...    ...
839980  290862 2020-06-30      2
841601  317427 2015-06-30      2
841602  317427 2015-09-30      2
842525  330227 2019-03-31      2
842526  330227 2019-06-30      2

[482 rows x 3 columns]


In [40]:
#How much collumn and wich one differ among duplicated ( gvkey, datadate)

# 1) Find all rows where (gvkey, datadate) is duplicated
dup_rows = compustat[compustat.duplicated(subset=['gvkey', 'datadate'], keep=False)]

# 2) Group those rows by (gvkey, datadate)
grouped = dup_rows.groupby(['gvkey', 'datadate'])

# 3) Build a summary listing, for each duplicate group, which columns differ
diff_list = []
for (gvkey, datadate), grp in grouped:
    # Count distinct values per column within this group
    nunique_per_col = grp.nunique()
    # Keep columns where there is more than 1 unique value
    cols_that_differ = nunique_per_col[nunique_per_col > 1].index.tolist()
    diff_list.append({
        'gvkey': gvkey,
        'datadate': datadate,
        'num_columns_differ': len(cols_that_differ),
        'columns_that_differ': cols_that_differ
    })

# 4) Turn that into a DataFrame
diff_summary = pd.DataFrame(diff_list)

# 5) Display the summary
print(diff_summary.head(10))


   gvkey   datadate  num_columns_differ  \
0   1076 1995-03-31                  10   
1   1117 1990-03-31                   3   
2   1161 1987-03-31                   9   
3   1224 2013-12-31                  21   
4   1224 2014-03-31                  10   
5   1224 2014-06-30                  10   
6   1263 2024-03-31                  36   
7   1263 2024-06-30                  35   
8   1263 2024-09-30                  35   
9   1263 2024-12-31                  23   

                                 columns_that_differ  
0  [fyearq, fqtr, fyr, cshpry, epsfiy, epsfxy, ep...  
1                                [fyearq, fqtr, fyr]  
2  [fyearq, fqtr, fyr, cshpry, epsfiy, epsfxy, ep...  
3  [fyearq, fqtr, fyr, cibegniy, citotaly, ciy, c...  
4  [fyr, cibegniy, citotaly, ciy, dpy, ibadjy, ib...  
5  [fyr, cibegniy, citotaly, ciy, dpy, ibadjy, ib...  
6  [fyearq, fqtr, fyr, cibegniy, cicurry, cipeny,...  
7  [fqtr, fyr, cibegniy, cicurry, cipeny, citotal...  
8  [fqtr, fyr, cibegniy, cicurr

In [60]:
compustat.head()

,gvkey,datadate,fyearq,fqtr,fyr,curcdq,acchgy,aolochy,apalchy,aqcy,...,txwy,xidocy,xidoy,xinty,xiy,xopry,xsgay,exchg,cik,fic
97,1004,1968-08-31,1968,1.0,5,USD,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,NaN,11.0,1750.0,USA
98,1004,1968-11-30,1968,2.0,5,USD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,1750.0,USA
99,1004,1969-02-28,1968,3.0,5,USD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,1750.0,USA
100,1004,1969-05-31,1968,4.0,5,USD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,1750.0,USA
101,1004,1969-08-31,1969,1.0,5,USD,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,NaN,11.0,1750.0,USA


In [62]:
column_names = compustat.columns.tolist()
print(f"Total columns: {len(column_names)}")
print(column_names)

Total columns: 92
['gvkey', 'datadate', 'fyearq', 'fqtr', 'fyr', 'curcdq', 'acchgy', 'aolochy', 'apalchy', 'aqcy', 'capxy', 'chechy', 'cibegniy', 'cicurry', 'cidergly', 'cimiiy', 'ciothery', 'cipeny', 'cisecgly', 'citotaly', 'ciy', 'cogsy', 'cshfdy', 'cshpry', 'cstkey', 'dilady', 'dilavy', 'dlcchy', 'dltisy', 'dltry', 'doy', 'dpcy', 'dpy', 'dvpy', 'dvy', 'epsfiy', 'epsfxy', 'epspiy', 'epspxy', 'esubcy', 'exrey', 'fiaoy', 'fincfy', 'fopoxy', 'fopoy', 'ibadjy', 'ibcomy', 'ibcy', 'ibmiiy', 'iby', 'intpny', 'invchy', 'ivacoy', 'ivchy', 'ivncfy', 'ivstchy', 'miiy', 'niy', 'nopiy', 'oancfy', 'oepsxy', 'oiadpy', 'oibdpy', 'opepsy', 'piy', 'prstkcy', 'rdipy', 'recchy', 'revty', 'saley', 'sivy', 'spiy', 'sppey', 'sppivy', 'sstky', 'stkcoy', 'txachy', 'txbcofy', 'txbcoy', 'txdcy', 'txpdy', 'txty', 'txwy', 'xidocy', 'xidoy', 'xinty', 'xiy', 'xopry', 'xsgay', 'exchg', 'cik', 'fic']
